In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = r'C:\Users\TH\Desktop\NN\cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

def load_data(filename):#从文件里读取数据
    with open(filename,'rb') as f:
        data =  pickle.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
    
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            for item,label in zip(data,labels):
                if label in [0,1]:#找到数据集中所有标签为0,1的函数并且放入一个新的数组中，shuffle_data将这些数据打乱顺序
                    all_data.append(item)
                    all_labels.append(label)
        self._data = np.vstack(all_data)
        self._data = self._data /127.5 -1 #归一化，很重要，数字太大实现不了
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data
            
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)#混排
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):#返回batch_size个样本
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

    
    
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

#batch_data,batch_labels = train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)  #测试CifarData类可以正常工作



x = tf.placeholder(tf.float32,[None,3072])
y = tf.placeholder(tf.int64,[None])    

w = tf.get_variable('w',[x.get_shape()[-1],1],initializer=tf.random_normal_initializer(0,1))#3072*1
b = tf.get_variable('b',initializer=tf.constant(0.0))#1

y_ = tf.matmul(x,w)+b#[None,3072]*[3072,1] = [None,1]
p_y_1 = tf.nn.sigmoid(y_) #[None,1]

y_reshaped = tf.reshape(y,(-1,1)) #[None,1]
y_reshaped_float32 = tf.cast(y_reshaped,tf.float32)
loss = tf.reduce_mean(tf.square(y_reshaped_float32 - p_y_1))

predict = p_y_1 > 0.5
correct_prediction = tf.equal(tf.cast(predict,tf.int64),y_reshaped)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    init.run()
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,accu_val,_ = sess.run([loss,accuracy,train_op],feed_dict={x:batch_data,y:batch_labels})
        
        if i%500 == 0 :
            print('[Train] step',i,' loss=',loss_val,' acc=',accu_val)
            
        if i%5000==0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data,test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('Test step',i+1,' acc=',test_acc)
            

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']
(10000, 3072)
(10000,)
(2000, 3072)
(2000,)
[Train] step 0  loss= 0.49488908  acc= 0.5
(2000, 3072)
(2000,)
Test step 1  acc= 0.36949999999999994
[Train] step 500  loss= 0.29198506  acc= 0.7
[Train] step 1000  loss= 0.22012739  acc= 0.75
[Train] step 1500  loss= 0.2999975  acc= 0.7
[Train] step 2000  loss= 0.34842578  acc= 0.65
[Train] step 2500  loss= 0.29999796  acc= 0.7
[Train] step 3000  loss= 0.14479968  acc= 0.85
[Train] step 3500  loss= 0.24730101  acc= 0.75
[Train] step 4000  loss= 0.22841664  acc= 0.75
[Train] step 4500  loss= 0.14504322  acc= 0.85
[Train] step 5000  loss= 0.21497531  acc= 0.75
(2000, 3072)
(2000,)
Test step 5001  acc= 0.794
[Train] step 5500  loss= 0.24873868  acc= 0.75
[Train] step 6000  loss= 0.19423775  acc= 0.8
[Train] step 6500  loss= 0.37991396  acc= 0.6
[Train] step 7000  loss= 0.15006858  acc= 0.85
[Train] step 7500  loss= 0.20